In [1]:
import json
import pandas as pd
from time import time
from connections import get_mongo_client
from my_utils import insert_into_mssql, load_my_columns, \
                        get_numbers,map_dff_to_my_columns, max_number_find


In [2]:
start = time()
write_to_table='bronze.katm_077_client'
columns_file = 'katm_077_client_fields.txt'
columns = load_my_columns(columns_file)
# max_num = max_number_find(write_to_table)

In [3]:
columns

['_id',
 'number',
 'pinfl',
 'live_address',
 'registration_district',
 'birth_date',
 'inn',
 'client_type',
 'founders',
 'oked',
 'email_change',
 'katm_sir',
 'juridical status',
 'name',
 'live_address_change',
 'registration_address',
 'email',
 'registration_address_change']

In [4]:
client = client = get_mongo_client()
db = client['task']
task_collection = db['task']
query = {
    'data.katm_077.return.data.client': {'$exists': True}
    ,'number': {'$eq': 7090166}
    # ,'number': {'$gt': max_num}
    # ,'number': {'$gt': 5000000, '$lt': 5001000}
    # ,'number': {'$in': numbers}
    
}
projection = {
    'number': 1,
    'data.katm_077.return.data.client': 1
}

In [5]:
docs = task_collection.find(query, projection)

In [5]:
doc = task_collection.find_one(query, projection)

In [6]:
doc

{'_id': ObjectId('670a33cc588a4631b21a4006'),
 'number': 7090166,
 'data': {'katm_077': {'return': {'data': {'dynamics_of_scoring_ball': {'periods': [{'period': '11.2023',
        'score_point': '232',
        'score_date': '2023-11-03 14:54'},
       {'period': '12.2023',
        'score_point': '241',
        'score_date': '2023-12-08 13:58'},
       {'period': '01.2024',
        'score_point': '246',
        'score_date': '2024-01-07 14:08'},
       {'period': '02.2024',
        'score_point': '239',
        'score_date': '2024-02-07 19:32'},
       {'period': '03.2024',
        'score_point': '226',
        'score_date': '2024-03-01 16:20'},
       {'period': '04.2024',
        'score_point': '246',
        'score_date': '2024-04-03 23:45'},
       {'period': '07.2024',
        'score_point': '251',
        'score_date': '2024-07-12 12:04'},
       {'period': '09.2024',
        'score_point': '167',
        'score_date': '2024-09-22 23:19'},
       {'period': '10.2024',
        'sco

In [6]:
for doc in docs:
    rows = []
    _id = str(doc.get('_id')) 
    number = doc.get('number')
    fields = doc.get('data', {}).get('katm_077', {}).get('return', {}).get('data', {}).get('client', {})
    if isinstance(fields, dict):
        fields = [fields]
    elif fields is None:
        fields = []
    for field in fields:
        row = {'_id': _id, 'number': number, **field}
        rows.append(row)

    df = pd.DataFrame(rows)
    final_df = map_dff_to_my_columns(df, columns)
    insert_into_mssql(final_df, write_to_table)
end = time()
print(f"script took {end-start} seconds")

script took 120.79887795448303 seconds


In [27]:
number

7090166